In [2]:
import enum
import hashlib
import re
from typing import Any, Self
from pydantic import (
    BaseModel,
    EmailStr,
    Field,
    field_serializer,
    field_validator,
    model_serializer,
    model_validator,
    SecretStr,
)

In [3]:
VALID_PASSWORD_REGEX = re.compile(r"^(?=.*[a-z])(?=.*[A-Z])(?=.*\d).{8,}$")
VALID_NAME_REGEX = re.compile(r"^[a-zA-Z]{2,}$")

In [4]:
class Role(enum.IntFlag):
    User = 0
    Author = 1
    Editor = 2
    Admin = 4
    SuperAdmin = 8

In [ ]:
class User(BaseModel):
    name: str = Field(examples=["Arjan"])
    email: EmailStr = Field(
        examples=["kaiquelima@alunos.utfpr.edu.br"],    # manda um exemplo pela função Field
        description="Endereço de email do usuário",     # descrição do campo
        frozen=True,                                    # só seta, não altera
    )

    # quando printar a senha, mostra *****
    password: SecretStr = Field(
        examples=["kai123"], description="Senha do usuário", exclude=True
    )

    role: Role = Field(
        default=None, description="Cargo do usuário"
    )

    # adição de um campo obrigatório
    especie : str = Field(
        default=None, description="Espécie do usuário", examples=["Humano", "Robô", "Alienígena"]
    )

    # cargo prático - gênero e idade
    genero: str = Field(
        default=None, description="Gênero do usuário", examples=["Masculino", "Feminino", "Outro"]
    )

    idade: int = Field(
        description="Idade do usuário"
    )

    # =-=-=-=-=-=- FIELD VALIDATOR =-=-=-=-=-=
    @field_validator("name")
    @classmethod
    def validate_name(cls, v: str) -> str:  # pega o nome e o retorna
        if not VALID_NAME_REGEX.match(v):   # passa pelo filtro do regex
            raise ValueError(               # se erro, retorna mensagem
                "Nome inválido, deve conter apenas letras e ter pelo menos 2 caracteres"
            )
        return v                            # retorna o nome validado

    # role validator
    @field_validator("role", mode="before")
    @classmethod
    def validate_role(cls, v: int | str | Role) -> Role:
        # dicionário que mapeia tipos para funções de conversão
        op = {
            int: lambda x: Role(x),      # se for int, converte para role usando o valor
            str: lambda x: Role[x],      # se for string, busca o role pelo nome
            Role: lambda x: x            # se já for role, retorna como está
        }

        try:
            return op[type(v)](v) # pega o tipo do valor recebido e aplica a função correspondente
        except (KeyError, ValueError):
            raise ValueError(
                f"Role is invalid, please use one of the following: {', '.join([x.name for x in Role])}"
            )

    # validação do campo espécie
    @field_validator("especie")
    @classmethod
    def validate_especie(cls, v: str) -> str:
        if not v:
            raise ValueError("Espécie é um campo obrigatório")
        if v not in ["Humano", "Robô", "Alienígena"]:
            raise ValueError("Espécie inválida, deve ser uma das seguintes: Humano, Robô, Alienígena")

        return v

    # validação do campo gênero
    @field_validator("genero")
    @classmethod
    def validate_genero(cls, v: str) -> str:
        if not v:
            raise ValueError("Gênero é um campo obrigatório")
        if v not in ["Masculino", "Feminino", "Outro"]:
            raise ValueError("Gênero inválido, deve ser um dos seguintes: Masculino, Feminino, Outro")

        return v

    # validação do campo idade
    @field_validator("idade")
    @classmethod
    def validate_idade(cls, v:int) -> int:
        if v < 0:
            raise ValueError("Idade não pode ser negativa")
        if v < 18:
            raise ValueError("Idade inválida, deve ser maior ou igual a 18 anos")
        if v > 120:
            raise ValueError("Idade inválida, deve ser menor ou igual a 120 anos")

        return v


    # =-=-=-=-=-=- MODEL VALIDATOR =-=-=-=-=-=
    @model_validator(mode="before")
    @classmethod
    def validate_user(cls, v: dict[str, Any]) -> dict[str, Any]:
        # verifica se os campos obrigatórios estão presentes
        if "name" not in v or "password" not in v:
            raise ValueError("Nome e senha são campos obrigatórios")

        # verifica se o nome não está contido na senha (case-insensitive)
        if v["name"].casefold() in v["password"].casefold():
            raise ValueError("Senha não pode conter o nome do usuário")

        # verifica se o email é a senha
        if v["email"].casefold() in v["password"].casefold():
            raise ValueError("Senha não pode conter o email do usuário")

        # valida a senha usando regex
        if not VALID_PASSWORD_REGEX.match(v["password"]):
            raise ValueError(
                "Senha inválida, deve conter pelo menos 8 caracteres, uma letra maiúscula, uma letra minúscula e um dígito"
            )

        # hash da senha usando SHA-256 antes de armazenar
        v["password"] = hashlib.sha256(v["password"].encode()).hexdigest()
        return v

    @model_validator(mode="after")
    def validate_user_post(self, v: Any) -> Self:
        if self.role == Role.Admin and self.name != "Kaique":
            raise ValueError("Only Kaique can be an admin")
        return self

    # =-=-=-=-=-=- SERIALIZER =-=-=-=-=-=
    # o field_serializer permite que você serialize o campo de uma maneira personalizada
    @field_serializer("role", when_used="json")
    @classmethod
    def serialize_role(cls, v) -> str:
        return v.name # vai serializar o campo role como uma string

    @field_serializer("especie", when_used="json")
    @classmethod
    def serialize_especie(cls, v: str) -> str:
        return v.lower()  # capitaliza a primeira letra da espécie

    @field_serializer("genero", when_used="json")
    @classmethod
    def serialize_genero(cls, v: str) -> str:
        return v.lower()  # capitaliza a primeira letra do gênero

    @field_serializer("idade", when_used="json")
    @classmethod
    def serialize_idade(cls, v: int) -> str:
        return str(v) # converte a idade para string

    # como os usuários podem ser serializados
    @model_serializer(mode="wrap", when_used="json")
    def serialize_user(self, serializer, info) -> dict[str, Any]:
        if not info.include and not info.exclude: # se não houver campos incluídos ou excluídos
            return {
                "name": self.name,
                "role": self.role.name,
                "email": self.email,
                "especie": self.especie.lower()
                } # retorna um dicionário com os campos
        return serializer(self)

In [16]:
good_data={
            "name": "Kaique",
            "password": "Password123",
            "email": "kaiquelima@alunos.utfpr.edu.br",
            "role": "Admin",
            "especie": "Humano",
            "genero": "Masculino",
            "idade": 20,
        }
user = User.model_validate(good_data)

if user:
    print(
        "The serializer that returns a dict:",
        user.model_dump(),
        sep="\n",
        end="\n\n",
    )
    print(
        "The serializer that returns a JSON string:",
        user.model_dump(mode="json"),
        sep="\n",
        end="\n\n",
    )
    print(
        "The serializer that returns a json string, excluding the role:",
        user.model_dump(exclude=["role"], mode="json"),
        sep="\n",
        end="\n\n",
    )
    print("The serializer that encodes all values to a dict:", dict(user), sep="\n")

The serializer that returns a dict:
{'name': 'Kaique', 'email': 'kaiquelima@alunos.utfpr.edu.br', 'role': <Role.Admin: 4>, 'especie': 'Humano', 'genero': 'Masculino', 'idade': 20}

The serializer that returns a JSON string:
{'name': 'Kaique', 'role': 'Admin', 'email': 'kaiquelima@alunos.utfpr.edu.br', 'especie': 'humano'}

The serializer that returns a json string, excluding the role:
{'name': 'Kaique', 'email': 'kaiquelima@alunos.utfpr.edu.br', 'especie': 'humano', 'genero': 'masculino', 'idade': '20'}

The serializer that encodes all values to a dict:
{'name': 'Kaique', 'email': 'kaiquelima@alunos.utfpr.edu.br', 'password': SecretStr('**********'), 'role': <Role.Admin: 4>, 'especie': 'Humano', 'genero': 'Masculino', 'idade': 20}
